# 14. Genetic Algorithm
 - 진화의 과정을 모방하는 방법

## 유전자 알고리즘의 절차 
 1. 정책을 유전자 형태로 표현
 2. 여러 번 시도하여 결과(return)가 좋은 정책을 선택
 3. 결과가 좋은 정책에 돌연변이를 추가
 4. 반복


## 유전자 알고리즘 vs. 강화학습
 - 구현이 상대적으로 간단
 - 가치를 추정하는 과정이 없음 
 - 복잡한 정책을 근사(approximation)하는 방법이 없음
 - 탐색 과정이 맹목적

### 예시
<a href='https://www.youtube.com/watch?v=Yr_nRnqeDp0'> 예시 </a>

In [1]:
import random
import numpy

## 주사위 

In [2]:
def dice():
    return random.randint(1,3)

In [3]:
dice()

3

In [4]:
dice()

3

In [5]:
def blackjack(policy):
    episode = []
    dealer = dice()
    dealer_go = True
    
    player = dice()
    player_go = True
    
    while dealer_go or player_go:
        
        # 딜러는 3점 미만이면 무조건 주사위 굴림
        dealer_go = dealer_go and dealer < 3
        
        # 플레이어는 정책이 1이면 주사위 굴림, 아니면 패스   
        player_go = player_go and policy[player - 1, dealer - 1]
        episode.append((player, dealer, int(player_go)))        
        
        if dealer_go:
            dealer += dice()
        
        if player_go:  
            player += dice()

        # 4점이 넘어가면 버스트
        if player > 4 and dealer > 4:
            return 0, episode  # 무승부
        elif dealer > 4:
            return 1, episode  # 승리
        elif player > 4:
            return -1, episode  # 패배
        # 그 외의 경우에는 계속

    if player > dealer:
        result = 1  # 승리
    elif player == dealer:
        result = 0  # 무승부
    else:
        result = -1 # 패배
    return result, episode      

## 무작위 정책

In [6]:
numpy.random.choice(2, (4, 4))

array([[0, 0, 0, 1],
       [1, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1]])

In [7]:
policies = numpy.random.choice(2, (10, 4, 4))
policies[0]

array([[0, 1, 1, 1],
       [1, 1, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 1]])

## 시행

In [8]:
results = numpy.zeros(10)

for i in range(10):
    for _ in range(100):
        policy = policies[i]
        result, history = blackjack(policy)
        results[i] += result

In [9]:
results

array([-52., -28., -36., -64., -58., -49., -45., -48., -25., -66.])

## 결과가 가장 좋은 정책 선택

In [10]:
best = numpy.argmax(results)
best

8

## 후손 생성

In [11]:
mother = policies[best]
policies = []
for i in range(9):
    child = numpy.array(mother)  # 새로운 정책
    
    x = random.randint(0, 3)
    y = random.randint(0, 3)
    child[x, y] = 1 - child[x, y]  # 무작위로 한 가지 정책을 바꿈
    policies.append(child)
policies.append(mother)

In [12]:
policies[8]

array([[0, 1, 1, 0],
       [1, 1, 1, 1],
       [0, 1, 1, 1],
       [1, 0, 0, 1]])